<a href="https://colab.research.google.com/github/SeongDahun/RecsysTutorial/blob/main/CTGAN_Credit_anomal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In this tutorial we use the defaul version of the CTGAN:
https://github.com/sdv-dev/CTGAN

We also need to install table evaluator to evaluate the quality of the generated synthetic data.

In [1]:
!pip install ctgan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.8 MB/s eta 0:00:00


In [2]:
!pip install table_evaluator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 31.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.


Now we download a dataset that includes both continuous and discrete values. I chose a random dataset from Kaggle: https://www.kaggle.com/datasets/mirichoi0218/insurance

In [3]:
#!gdown 1CFLXcella3VFDzPJRGrfkGlYNOhf1RCQ #Insurance 데이터 불러오기 (아마 다른 사람 드라이브에서 다운로드?)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/data/creditcard.csv')

In [6]:
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [7]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [8]:
# 정상 데이터 추출
nomalies = data[data['Class'] == 0]
nomalies

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [9]:
# 정상 데이터 줄이는 경우
sample_nomalies = nomalies.sample(frac=0.1)
sample_nomalies.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 28432 entries, 82172 to 126133
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    28432 non-null  float64
 1   V1      28432 non-null  float64
 2   V2      28432 non-null  float64
 3   V3      28432 non-null  float64
 4   V4      28432 non-null  float64
 5   V5      28432 non-null  float64
 6   V6      28432 non-null  float64
 7   V7      28432 non-null  float64
 8   V8      28432 non-null  float64
 9   V9      28432 non-null  float64
 10  V10     28432 non-null  float64
 11  V11     28432 non-null  float64
 12  V12     28432 non-null  float64
 13  V13     28432 non-null  float64
 14  V14     28432 non-null  float64
 15  V15     28432 non-null  float64
 16  V16     28432 non-null  float64
 17  V17     28432 non-null  float64
 18  V18     28432 non-null  float64
 19  V19     28432 non-null  float64
 20  V20     28432 non-null  float64
 21  V21     28432 non-null  float64
 22

In [10]:
# 이상 데이터 추출
anomalies = data[data['Class'] == 1]
anomalies

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
6108,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
6329,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


Next, we define a list with column names for categorical variables. This list will be passed to the model so that the model can decide how to process these fields.

In [11]:
categorical_features = ['Time','Class']

#Model training

Next, we simply define an instance of CTGAN Synthesizer and call the fit method with the dataframe and the list of categorical variables.

We train the model for 300 epochs only as the discriminator and generator loss becomes quite low after these many epochs.

In [12]:
from ctgan import CTGAN

In [13]:
# 이상 데이터 학습
ctgan_anomal = CTGAN(verbose=True)
ctgan_anomal.fit(anomalies, categorical_features, epochs = 200)

/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'mi

Epoch 1, Loss G:  3.0912,Loss D:  0.0088
Epoch 2, Loss G:  3.3156,Loss D: -0.0327
Epoch 3, Loss G:  3.0740,Loss D: -0.0827
Epoch 4, Loss G:  2.8457,Loss D: -0.1204
Epoch 5, Loss G:  3.4079,Loss D: -0.1755
Epoch 6, Loss G:  2.9587,Loss D: -0.2339
Epoch 7, Loss G:  3.1943,Loss D: -0.2956
Epoch 8, Loss G:  3.2716,Loss D: -0.3506
Epoch 9, Loss G:  3.3224,Loss D: -0.4399
Epoch 10, Loss G:  2.8177,Loss D: -0.4631
Epoch 11, Loss G:  3.1897,Loss D: -0.5109
Epoch 12, Loss G:  2.8304,Loss D: -0.5552
Epoch 13, Loss G:  3.0551,Loss D: -0.5429
Epoch 14, Loss G:  2.9087,Loss D: -0.5418
Epoch 15, Loss G:  2.7280,Loss D: -0.5568
Epoch 16, Loss G:  2.7960,Loss D: -0.5194
Epoch 17, Loss G:  3.0083,Loss D: -0.4650
Epoch 18, Loss G:  2.6843,Loss D: -0.4062
Epoch 19, Loss G:  2.6469,Loss D: -0.2317
Epoch 20, Loss G:  2.5467,Loss D: -0.2085
Epoch 21, Loss G:  2.4115,Loss D: -0.2076
Epoch 22, Loss G:  2.4225,Loss D: -0.0401
Epoch 23, Loss G:  2.4879,Loss D: -0.0367
Epoch 24, Loss G:  2.4192,Loss D: -0.0420
E

#Synthetic data generation

In [14]:
# 이상 데이터의 증강 데이터 생성
synthetic_anomal = ctgan_anomal.sample(492)
synthetic_anomal

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,60353.0,-5.890499,8.782787,-5.102155,7.511173,-1.675120,-6.065489,-3.431684,-6.165284,-2.443822,...,-0.364409,-0.028779,0.141589,0.301745,0.664994,0.115549,0.683914,-0.080392,58.622529,1
1,87883.0,2.090472,14.798802,-7.071716,4.338972,3.679079,-3.288600,-7.496640,2.741420,-1.681104,...,-0.391807,1.799539,-0.074380,1.223307,-0.813682,-0.319507,0.148993,-0.089985,42.880207,1
2,146026.0,-4.297712,-2.092887,-1.490517,0.993092,0.923453,-1.298138,-13.162487,-2.291086,-1.195603,...,-1.377850,-0.101650,0.666764,-0.196570,-0.378717,-0.090350,0.292596,0.680109,6.295407,1
3,46057.0,-6.685845,0.147343,-18.928414,6.803116,-5.295701,-2.549145,-1.939436,0.596061,-0.268565,...,-15.717219,1.400703,-0.688312,0.902852,0.934412,-0.122756,0.255887,-0.452672,362.353320,1
4,77202.0,-2.644642,-0.942558,-4.646055,2.207903,2.123424,-0.174635,-6.819609,0.433411,-2.267398,...,-0.084584,0.818123,-0.375247,0.858434,0.596546,0.344103,-0.652450,0.011629,34.680983,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,155548.0,1.131228,0.351626,-15.142389,7.824148,-0.699144,-5.080878,-3.498656,0.685994,-3.517118,...,-0.740654,-0.830535,0.274620,-0.536981,-0.187543,-0.180654,0.928378,-0.023151,1051.234112,1
488,94364.0,-3.411883,0.564494,3.181939,0.049994,-15.323412,-2.979540,-28.669318,1.149798,-1.249859,...,-0.183026,-0.506764,0.573914,0.801791,0.330767,-0.129013,0.162143,0.468026,698.466040,1
489,37167.0,-17.397692,5.325914,2.990439,3.554739,-0.127576,-3.922823,-45.343269,19.344391,-2.768140,...,0.572355,-1.504252,-0.032755,0.975151,0.234411,-0.787440,1.265930,0.602204,95.359954,1
490,144808.0,1.620829,6.324700,0.669030,6.794294,-4.329114,-2.884961,-12.416131,-0.546826,0.814790,...,0.355244,0.211076,-0.377047,0.715462,0.154764,0.295781,0.112880,0.819376,691.913585,1


In [15]:
# 정상 데이터 줄이는 경우
  # 정상 줄인 데이터 + 이상 증강 데이터
extended_data = pd.concat([sample_nomalies, synthetic_anomal], ignore_index=True)

#Evaluation

In [17]:
from table_evaluator import TableEvaluator

In [ ]:
# # 이상 데이터만 늘릴 때
#     # 이상 데이터 원본과 이상 데이터의 증강 데이터 비교
# print(anomalies.shape, synthetic_anomal.shape)
# table_evaluator_anomal =  TableEvaluator(anomalies, synthetic_anomal, cat_cols= categorical_features)

# # table_evaluator_anomal.visual_evaluation()

In [18]:
# 정상 데이터 줄여서
    # 정상 줄인 데이터 vs 정상 줄인+이상 증강 데이터 비교
print(sample_nomalies.shape, extended_data.shape)
table_evaluator_anomal =  TableEvaluator(sample_nomalies, extended_data, cat_cols= categorical_features)

(28432, 31) (28924, 31)


In [ ]:
table_evaluator_anomal.evaluate(target_col='Time') # 한 컬럼에 대해서만 유사도 검사. 범주형 데이터만 가능